# Fire Event Capture and Visualization using Sentinel-2 Imagery
This notebook showcases how to use **Google Earth Engine (GEE)** and  **Sentinel-2** data to analyze wildfire events based on **MODIS/VIIRS** fire data. We will load fire event data, extract relevant imagery, visualize the affected area, and download satellite images.

## How to set up?

1. Open Script In Colab: https://colab.research.google.com/github/gunelaliyevaa/wildfire-detection-using-satellite-imagery/blob/main/scripts/image_extraction_script.ipynb
2. Clone the repository
3. Install dependencies
4. Import the dataset processing library

### Set Up
Clone the repository and install necessary dependencies to get started.


In [ ]:
!git clone https://github.com/gunelaliyevaa/wildfire-detection-using-satellite-imagery.git

Change the working directory to the actual project folder.

In [ ]:
%cd wildfire-detection-using-satellite-imagery

Install the custom dataset processing library (dataset_processing_lib) from the cloned repository.

In [ ]:
!pip install ./dataset_processing_lib

Import the modules used in the project including dataset_processing_lib.

In [ ]:
import os
import ee
import pandas as pd
import requests
import dataset_processing_lib as dpl
from concurrent.futures import ThreadPoolExecutor, as_completed

The variable `collection` defines the satellite image collection that we will use from Google Earth Engine (GEE). _COPERNICUS/S2_SR_HARMONIZED_ refers to the Sentinel-2 Surface Reflectance dataset, which provides high-resolution satellite imagery *corrected for atmospheric conditions*(?).
<br>
<br>
Sentinel-2 is widely used for wildfire detection because it captures visible, near-infrared (NIR), and shortwave infrared (SWIR) bands, which help in detecting burn scars and active fire areas. There are many other image collections available in GEE that you can utilize depending on your needs. Visit __[Google Earth Engine Data Catalog](https://developers.google.com/earth-engine/datasets)__ to search for other available datasets.

The variable `project` defines the Google Earth Engine project ID used for authentication and resource access. Google Earth Engine requires users to specify a project to track resource usage, API calls, and storage. The project ID is linked to your Google Cloud Platform (GCP) account.
<br>
<div class="alert alert-block alert-info">
<b> How to create a new Google Cloud Project? </b> <br>
1. Go to <a href = "https://console.cloud.google.com/"> Google Cloud Console</a> <br>
2. Click "Select a project" → "New Project". <br>
3. Enter your project credentials. <br>
4. Copy your Project ID and replace 'wildfire-detection-ml' if necessary.
</div>



 `data_path` variable stores the URL of the fire event dataset (CSV file), which contains geospatial and satellite information about fire incidents. The csv files are obtained from NASA’s Fire Information for Resource Management System (FIRMS). It provides near real-time fire detection alerts based on satellite observations. The data is collected using multiple Earth observation instruments,MODIS and VIIRS, each with different spatial and temporal resolutions. Data can be requested from __[NASA FIRMS](https://firms.modaps.eosdis.nasa.gov/download/)__ by specifying the area of interest, instrument, and a date range.

In [ ]:
collection = 'COPERNICUS/S2_SR_HARMONIZED'
project = 'wildfire-detection-ml'
data_path = 'https://raw.githubusercontent.com/gunelaliyevaa/wildfire-detection-using-satellite-imagery/main/data/raw/j2_viirs/2024_fire_nrt_J2V-C2_539017.csv'
output_folder = 'img_folder'

In [ ]:
df = pd.read_csv(data_path)
df.head()

In [ ]:
ee.Authenticate()
ee.Initialize(project=project)

---

In [45]:
def get_satellite_collection(longitude, latitude, start_date, end_date, buffer=0.02, bands=['B4', 'B3', 'B2']):
    """Fetches Sentinel-2 images for the given location and date range."""
    geometry = ee.Geometry.Rectangle([
        longitude - buffer,
        latitude - buffer,
        longitude + buffer,
       latitude + buffer
    ])

    # geometry = ee.Geometry.Point([longitude, latitude]).buffer(buffer).bounds()

    filtered_collection = (ee.ImageCollection(collection)
                           .filterBounds(geometry)
                           .filterDate(start_date, end_date)
                           .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 15)
                           .map(lambda img: img.divide(10000)))  # Normalize
    if filtered_collection.size().getInfo() == 0:
        return None, None

    return filtered_collection.median().select(bands).clip(geometry), geometry

---

In [42]:
# import these functions from utils
def generate_download_url(image, rectangle):
    """Generates a download URL for the processed image."""
    return image.getThumbURL({
        'min': 0,
        'max': 0.5,
        'dimensions': 512,
        'format': 'png',
        'bands': ['B4', 'B3', 'B2'],
        'region': rectangle
    })

def download_image(url):
    """Downloads an image from the given URL and returns the content as bytes."""
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        return response.content
    except requests.RequestException as e:
        print(f"Failed to download image: {e}")
        return None

def save_image(image_content, output_dir, filename):
    """Saves the given image content to the specified directory with the given filename."""
    if image_content is None:
        print(f"Skipping save: No content for {filename}")
        return

    os.makedirs(output_dir, exist_ok=True)
    file_path = os.path.join(output_dir, filename)

    try:
        with open(file_path, 'wb') as f:
            f.write(image_content)
        print(f"Saved: {file_path}")
    except OSError as e:
        print(f"Failed to save {filename}: {e}")

---

In [43]:
def process_single_event(row, output_dir):
    """Processes a single fire event and downloads the corresponding image."""
    try:
        longitude, latitude = row['longitude'], row['latitude']
        acq_date = pd.to_datetime(row['acq_date'])

        start_date = (acq_date - pd.DateOffset(days=1)).strftime('%Y-%m-%d')
        end_date = (acq_date + pd.DateOffset(days=1)).strftime('%Y-%m-%d')

        image, geometry = get_satellite_collection(
            longitude=longitude,
            latitude=latitude,
            start_date=start_date,
            end_date=end_date,
        )

        if not image:
            return None

        filename = f"{row['latitude']}_{row['longitude']}_{acq_date.date()}.png"
        output_path = os.path.join(output_dir, filename)

        url = generate_download_url(image, geometry)
        image_content = download_image(url)  # Now returns the image content (bytes)

        if image_content:
            save_image(image_content, output_dir, filename)
            return output_path

        return None

    except Exception as e:
        print(f"Error processing event: {str(e)}")
        return None

def process_event_batch(fire_df, output_dir, max_workers=5):
    """Processes fire events concurrently using a thread pool."""
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(process_single_event, row, output_dir)
            for _, row in fire_df.iterrows()
        ]
        return [f.result() for f in as_completed(futures)]

---

In [ ]:
process_event_batch(df, output_folder, max_workers=5)


The command `!zip -r my_folder.zip my_folder` creates a compressed ZIP archive of the folder `my_folder`. This will be helpful for downloading the extracted images to your local machine in a single folder.

In [ ]:
# !zip -r MODIS_Images_2024.zip MODIS_Images_2024

In [ ]:
# import shutil

# Specify the folder to remove
# folder_to_remove = 'SUOMI_VIIRS_IMAGES_2023'

# Remove the folder and its contents
# shutil.rmtree(folder_to_remove)
# print(f"Folder '{folder_to_remove}' and its contents have been removed.")